In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

In [33]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

# Read the sample data from CSV
df = pd.read_csv("dataa.csv")

# Convert date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Label encode categorical variables
label_encoder = LabelEncoder()
df['Shift'] = label_encoder.fit_transform(df['Shift'])
df['Line'] = label_encoder.fit_transform(df['Line'])
df['Product'] = label_encoder.fit_transform(df['Product'])

# Define function for extrapolation
def extrapolate(df, n):
    # Fit a linear regression model for each unique combination of Shift, Line, and Product
    models = {}
    for key, group in df.groupby(['Shift', 'Line', 'Product']):
        X = group[['CycleTime', 'Downtime', 'MaintenanceCost']]
        y = group['Yield']
        model = LinearRegression().fit(X, y)
        models[key] = model
    
    # Extrapolate for each unique combination of Shift, Line, and Product
    result = []
    for key, model in models.items():
        for i in range(n):
            cycle_time = df['CycleTime'].mean()
            downtime = df['Downtime'].mean()
            maintenance_cost = df['MaintenanceCost'].mean()
            data = [[cycle_time, downtime, maintenance_cost]]
            prediction = model.predict(data)[0]
            record = list(key) + [prediction, 0, 0, cycle_time, downtime, maintenance_cost]
            result.append(record)
    
    # Create a new DataFrame for the extrapolated data
    cols = ['Shift', 'Line', 'Product', 'Yield', 'Scrap', 'Rework', 'CycleTime', 'Downtime', 'MaintenanceCost']
    result_df = pd.DataFrame(result, columns=cols)
    
    # Convert categorical variables back to strings
    result_df['Shift'] = label_encoder.inverse_transform(result_df['Shift'])
    result_df['Line'] = label_encoder.inverse_transform(result_df['Line'])
    result_df['Product'] = label_encoder.inverse_transform(result_df['Product'])
    
    return result_df

# Define start and end date
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 1, 1)

# Extrapolate for each month between start and end date
result = []
for month in pd.date_range(start=start_date, end=end_date, freq='MS'):
    month_df = df[df['Date'] <= month]
    month_result = extrapolate(month_df, 1000)
    month_result['Date'] = month
    result.append(month_result)

# Concatenate the extrapolated data for each month into a single DataFrame
extrapolated_df = pd.concat(result)

# Convert date column back to string format
extrapolated_df['Date'] = extrapolated_df['Date'].dt.strftime('%Y-%m-%d')

# Save the extrapolated data to CSV
extrapolated_df.to_csv("extrapolated_data.csv", index=False)


In [36]:
Ed= pd.read_csv("extrapolated_data.csv")
Ed

,Shift,Line,Product,Yield,Scrap,Rework,CycleTime,Downtime,MaintenanceCost,Date
0,A,A,A,0.95,0,0,124.583333,64.166667,111.666667,2022-01-01
1,A,A,A,0.95,0,0,124.583333,64.166667,111.666667,2022-01-01
2,A,A,A,0.95,0,0,124.583333,64.166667,111.666667,2022-01-01
3,A,A,A,0.95,0,0,124.583333,64.166667,111.666667,2022-01-01
4,A,A,A,0.95,0,0,124.583333,64.166667,111.666667,2022-01-01
...,...,...,...,...,...,...,...,...,...,...
155995,A,C,D,0.94,0,0,124.583333,64.166667,111.666667,2023-01-01
155996,A,C,D,0.94,0,0,124.583333,64.166667,111.666667,2023-01-01
155997,A,C,D,0.94,0,0,124.583333,64.166667,111.666667,2023-01-01
155998,A,C,D,0.94,0,0,124.583333,64.166667,111.666667,2023-01-01


In [37]:
Ed.isnull().sum()

Shift              0
Line               0
Product            0
Yield              0
Scrap              0
Rework             0
CycleTime          0
Downtime           0
MaintenanceCost    0
Date               0
dtype: int64